In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df=pd.read_csv("water_potability.csv")
df1=df.copy()

In [8]:
z=["ph","Sulfate","Trihalomethanes"]
for col in z:
    df[col].replace(np.nan, df[col].mean(), inplace=True)

In [10]:
df.drop(3014,inplace=True)

In [11]:
df_1 =df[df['Potability']==1]
df_0 = df[df['Potability']==0]
# no_water = df.dropna()
cols = list(df_1.columns)

In [14]:
def change(df_1,column, upper_limit,lis,lower_limit):
    for i in range(len(df_1)):
        x = df_1.iloc[i:i+1][column].values[0]
        if (x>=upper_limit):
            lis[i] = 0
        if(column=='ph'):
            if(x<=lower_limit):
                lis[i] = 0
    return lis

In [13]:
lis = df_1['Potability'].values
lis1 = df_0['Potability'].values

In [15]:

for i in cols[:-1]:
    Q1 = df_1[i].quantile(0.25)
    Q3 =df_1[i].quantile(0.75)
    iqr = Q3 - Q1
    upper_limit = Q3 + 1.5* iqr
    lower_limit = Q1 - 1.5 * iqr
    print(i,upper_limit,lower_limit)
    lis = change(df_1,i,upper_limit, lis, lower_limit)
# print(len(lis))

ph 9.922433443185925 4.209458530820333
Hardness 283.5127538042832 108.82119802510387
Solids 46429.61356370653 -2787.3920824244906
Chloramines 11.356953261826861 2.9364417441007067
Sulfate 417.44038994875245 250.4204810090075
Conductivity 668.981243898836 176.11368975582258
Organic_carbon 22.83976790942959 5.550374644552143
Trihalomethanes 107.3018632545189 26.676779263653483
Turbidity 6.127557485528579 1.8129204506468555


In [17]:
dff=pd.DataFrame() 
for i in cols:
    dff[i] = df_1[i]


In [18]:
dff['Potability'] = lis

In [19]:
final_df = pd.concat([df_0, dff])
final_df1 = final_df[final_df['Potability']==1]
final_df2 = final_df[final_df['Potability']==0]

In [21]:
for i in cols:
    cx = df_0[i].max()
    print(cx)
    final_df2[i].fillna(float(cx), inplace = True)

13.999999999999998
304.23591211231667
61227.19600771213
12.653362021943142
460.1070689926536
753.3426195583046
28.30000000000001
120.03007700530677
6.739
0


C:\Users\mamid\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [24]:

lid = list(final_df1['Potability']) + list(final_df2['Potability'])


In [30]:
from  sklearn.preprocessing import PowerTransformer
px = PowerTransformer(method = 'box-cox')
x_trans = px.fit_transform(final_df1.iloc[:,:-1])
x1_trans = px.fit_transform(final_df2.iloc[:,:-1])


In [31]:
d1 = pd.DataFrame(x_trans, columns=cols[:-1])
d2 = pd.DataFrame(x1_trans, columns=cols[:-1])

In [32]:
fi1 = pd.concat([d1,d2],ignore_index=True)
fi1['Potability'] = lid

In [33]:
final_df = pd.concat([final_df1, final_df2],ignore_index=True)

In [47]:
# from sklearn.neural_network import MLPClassifier
# mlp = MLPClassifier(activation="relu", max_iter = 500, hidden_layer_sizes=300 )
# mlp.fit(x_train, y_train)
# y_pred = mlp.predict(x_test)

C:\Users\mamid\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [34]:
#split the dataset in Training set and Test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(fi1.iloc[:,:-1], fi1['Potability'], test_size = 0.25,random_state=400)


# XGBOOST


In [124]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score
x_train, x_test, y_train, y_test = train_test_split(fi1.iloc[:,:-1], fi1['Potability'], test_size = 0.25,random_state=299)
Xgb = XGBClassifier(max_depth=9, n_estimators = 102, n_jobs=1,colsample_bytree=0.7,reg_alpha=0,reg_lambda=0.5,gamma=0.1)
Xgb.fit(x_train,y_train)
y_pred = Xgb.predict(x_test)
c = float(accuracy_score(y_test,y_pred))
print(c)

0.8205128205128205


In [127]:
import pickle
pickle_out=open("clsxgb.pkl","wb")
pickle.dump(Xgb,pickle_out)
pickle_out.close()

# RANDOMFOREST

In [126]:
    from sklearn.ensemble import RandomForestClassifier
    x_train, x_test, y_train, y_test = train_test_split(fi1.iloc[:,:-1], fi1['Potability'], test_size = 0.25,random_state=292)
    rfc = RandomForestClassifier(n_estimators=100,max_features= 0.8, criterion="entropy",max_depth= 8,random_state=27);
    rfc.fit(x_train, y_train)
    y_pred = rfc.predict(x_test)
    a=accuracy_score(y_pred, y_test)
    print(a)

0.8290598290598291


In [128]:
import pickle
pickle_out=open("clsrf.pkl","wb")
pickle.dump(rfc,pickle_out)
pickle_out.close()

# LOGISTIC REGRESSION

In [115]:
from sklearn.linear_model import LogisticRegression
x_train, x_test, y_train, y_test = train_test_split(fi1.iloc[:,:-1], fi1['Potability'], test_size = 0.30098,random_state=353)
log_clf = LogisticRegression(random_state=190,solver='saga',max_iter=100)
log_clf.fit(x_train,y_train)
y_predl=log_clf.predict(x_test)
accuracy_score(y_predl, y_test)


0.6886409736308317

In [129]:
import pickle
pickle_out=open("clslr.pkl","wb")
pickle.dump(log_clf,pickle_out)
pickle_out.close()

In [125]:
# op={}
# # from sklearn.linear_model import LogisticRegression
# for i in range(400,500):
#     x_train, x_test, y_train, y_test = train_test_split(fi1.iloc[:,:-1], fi1['Potability'], test_size = 0.25,random_state=i)
#     Xgb = XGBClassifier(max_depth=9, n_estimators = 102, n_jobs=1,colsample_bytree=0.7,reg_alpha=0,reg_lambda=0.5,gamma=0.1)
#     Xgb.fit(x_train,y_train)
#     y_pred = Xgb.predict(x_test)
#     c = float(accuracy_score(y_test,y_pred))
#     op.update({c:i})
# for k,v in op.items():    
#     if k==max(list(op.keys())):
#         print(k,v)

0.811965811965812 463


<!-- import pickle
pickle_out=open("cls.pkl","wb")
pickle.dump(rfor_clf,pickle_out)
pickle_out.close() -->